## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-11-02-22-00 +0000,wsj,Onshore Yuan Strengthens After PBOC’s Currency...,https://www.wsj.com/finance/currencies/asian-c...
1,2025-12-11-02-16-41 +0000,nypost,"Andy Dick turned blue during apparent OD, says...",https://nypost.com/2025/12/10/us-news/andy-dic...
2,2025-12-11-02-16-17 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...
3,2025-12-11-02-13-06 +0000,cbc,Report alleges national museum CEO mistreated ...,https://www.cbc.ca/news/politics/museum-of-imm...
4,2025-12-11-02-13-00 +0000,wsj,"For Trump, the Warner Megadeal Talks Are All A...",https://www.wsj.com/business/media/for-trump-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,55
163,fed,26
303,new,23
88,venezuela,13
408,media,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
302,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...,141
45,2025-12-10-23-56-02 +0000,nypost,Trump says he wants a new owner for CNN as par...,https://nypost.com/2025/12/10/media/trump-says...,126
348,2025-12-10-09-00-00 +0000,cbc,Is Trump’s stark new security strategy the end...,https://www.cbc.ca/news/world/trump-world-orde...,118
361,2025-12-10-07-00-26 +0000,nyt,Trump Revives Attacks on Fed Members Before In...,https://www.nytimes.com/2025/12/09/us/politics...,118
28,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
302,141,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...
346,72,2025-12-10-09-34-00 +0000,wsj,Social-Media Ban Imposes Brave New World on Au...,https://www.wsj.com/world/social-media-ban-imp...
26,68,2025-12-11-00-41-04 +0000,nyt,Oil Tanker U.S. Seized Off Venezuela Has Faked...,https://www.nytimes.com/2025/12/10/us/politics...
230,55,2025-12-10-17-09-40 +0000,nypost,Machado’s daughter accepts Nobel Peace Prize i...,https://nypost.com/2025/12/10/world-news/macha...
266,53,2025-12-10-15-11-40 +0000,nypost,Husband charged with murdering Miss Switzerlan...,https://nypost.com/2025/12/10/world-news/husba...
273,48,2025-12-10-14-48-10 +0000,cbc,Bank of Canada holds key interest rate at 2.25...,https://www.cbc.ca/news/business/bank-of-canad...
200,44,2025-12-10-18-30-39 +0000,latimes,Democratic lawmakers say Paramount bid for War...,https://www.latimes.com/entertainment-arts/bus...
28,43,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...
160,41,2025-12-10-20-13-01 +0000,nypost,Nvidia builds AI chip-tracking software while ...,https://nypost.com/2025/12/10/business/nvidia-...
65,40,2025-12-10-23-21-00 +0000,wsj,More Republicans in the House and Senate are b...,https://www.wsj.com/politics/policy/some-repub...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
